In [ ]:

from pyspark.sql import SparkSession

# @hidden_cell
# This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_8eb7e5fdaa4240cba62b017a11bd6579(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', 'a163084eea8d49a9a8f6bfa978c7c788')
    hconf.set(prefix + '.username', '6bd8640672ce474a97df18170985958e')
    hconf.set(prefix + '.password', 'jE--F9s1_1#o*i}3')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_8eb7e5fdaa4240cba62b017a11bd6579(name)

spark = SparkSession.builder.getOrCreate()

# Please read the documentation of PySpark to learn more about the possibilities to load data files.
# PySpark documentation: https://spark.apache.org/docs/2.0.1/api/python/pyspark.sql.html#pyspark.sql.SparkSession
# The SparkSession object is already initalized for you.
# The following variable contains the path to your file on your Object Storage.
path_1 = "swift://aaaaa." + name + "/100kReviews.txt"



In [ ]:

stopwords = ["i","me","my","myself","we","our","ours","ourselves","you","your","yours","yourself","yourselves","he","him","his","himself","she","her","hers","herself","it","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this","that","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","above","below","to","from","up","down","in","out","on","off","over","under","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too","very","s","t","can","will","just","don","should","now","i'll","you'll","he'll","she'll","we'll","they'll","i'd","you'd","he'd","she'd","we'd","they'd","i'm","you're","he's","she's","it's","we're","they're","i've","we've","you've","they've","isn't","aren't","wasn't","weren't","haven't","hasn't","hadn't","don't","doesn't","didn't","won't","wouldn't","shan't","shouldn't","mustn't","can't","couldn't","cannot","could","here's","how's","let's","ought","that's","there's","what's","when's","where's","who's","why's","would"]

# convert all words to lowercase
# remove words that are too long or too short 
reviewsRdd = sc.textFile(path_1).map(lambda line: line.lower()).map(lambda line: " ".join(word for word in line.strip().split() if 25 > len(word) > 4))



In [ ]:

reviewsRdd = reviewsRdd.map(lambda line: " ".join(word for word in line.strip().split() if word not in stopwords))



In [ ]:

from pyspark.sql import SQLContext, Row
rowedReviewsRdd = reviewsRdd.map(lambda (data): Row(data = data))



In [ ]:

reviewsRddDF = sqlContext.createDataFrame(rowedReviewsRdd)



In [ ]:

from pyspark.ml import PipelineModel
from pyspark.ml.feature import CountVectorizer, Tokenizer
from pyspark.sql import Row



In [ ]:

tokenizer = Tokenizer(inputCol="data", outputCol="words")
countVectorizer = CountVectorizer(inputCol="words", outputCol="features")
pipeline = PipelineModel(stages=[tokenizer])



In [ ]:

model = countVectorizer.fit(pipeline.transform(reviewsRddDF))
result = model.transform(pipeline.transform(reviewsRddDF))



In [ ]:

modelToTrain = result.select("words","features")



In [ ]:

from pyspark.ml.clustering import LDA
modelToTrain.cache()



In [ ]:

lda = LDA(k = 2)



In [ ]:

trainedModel = lda.fit(modelToTrain)



In [ ]:

vocabArray = model.vocabulary



In [ ]:

for i in trainedModel.describeTopics(6).rdd.map(tuple).collect():
    print "Topic "+str(i[0])
    for j in i[1]:
        print vocabArray[j]
    print "\n"



In [ ]:

#The meanings of German result by order
# Not
# One
# Eat
# Were
# One
# something

